# 近似训练
:label:`sec_approx_train`

回顾我们在 :numref:`sec_word2vec` 中的讨论。skip-gram 模型的主要思想是使用 softmax 操作来计算基于给定中心词 $w_c$ 生成上下文词 $w_o$ 的条件概率，如 :eqref:`eq_skip-gram-softmax` 所示，其对应的对数损失为 :eqref:`eq_skip-gram-log` 的相反数。

由于 softmax 操作的特性，因为一个上下文词可能是词汇表 $\mathcal{V}$ 中的任何一个词，:eqref:`eq_skip-gram-log` 的相反数包含与整个词汇量大小相同的项的总和。因此，skip-gram 模型在 :eqref:`eq_skip-gram-grad` 和连续词袋模型在 :eqref:`eq_cbow-gradient` 中的梯度计算都包含这种求和。不幸的是，对于这种对大型词典（通常包含数十万或数百万个词）进行求和的梯度计算成本巨大！

为了减少上述计算复杂性，本节将介绍两种近似训练方法：*负采样*和*层次化 softmax*。由于 skip-gram 模型和连续词袋模型之间的相似性，我们将仅以 skip-gram 模型为例来描述这两种近似训练方法。

## 负采样
:label:`subsec_negative-sampling`

负采样修改了原始的目标函数。给定中心词 $w_c$ 的上下文窗口，任何（上下文）词 $w_o$ 来自这个上下文窗口的事实被认为是具有以下概率的事件：

$$P(D=1\mid w_c, w_o) = \sigma(\mathbf{u}_o^\top \mathbf{v}_c),$$

其中 $\sigma$ 使用 sigmoid 激活函数的定义：

$$\sigma(x) = \frac{1}{1+\exp(-x)}.$$
:eqlabel:`eq_sigma-f`

让我们通过最大化文本序列中所有这些事件的联合概率来训练词嵌入。具体来说，给定长度为 $T$ 的文本序列，用 $w^{(t)}$ 表示时间步 $t$ 处的词，并设上下文窗口大小为 $m$，考虑最大化联合概率

$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(D=1\mid w^{(t)}, w^{(t+j)}).$$
:eqlabel:`eq-negative-sample-pos`

然而，:eqref:`eq-negative-sample-pos` 仅考虑涉及正例的事件。结果，:eqref:`eq-negative-sample-pos` 中的联合概率只有当所有词向量等于无穷大时才最大化为 1。当然，这样的结果是没有意义的。为了让目标函数更有意义，*负采样*添加了从预定义分布中采样的负例。

设 $S$ 为上下文词 $w_o$ 来自中心词 $w_c$ 的上下文窗口的事件。对于涉及 $w_o$ 的这个事件，从预定义分布 $P(w)$ 中采样 $K$ 个*噪声词*，它们不是来自这个上下文窗口。设 $N_k$ 为噪声词 $w_k$（$k=1, \ldots, K$）不来自 $w_c$ 的上下文窗口的事件。假设涉及正例和负例的事件 $S, N_1, \ldots, N_K$ 是相互独立的。负采样将 :eqref:`eq-negative-sample-pos` 中仅涉及正例的联合概率重写为

$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)}),$$

其中条件概率通过事件 $S, N_1, \ldots, N_K$ 进行近似：

$$ P(w^{(t+j)} \mid w^{(t)}) =P(D=1\mid w^{(t)}, w^{(t+j)})\prod_{k=1,\ w_k \sim P(w)}^K P(D=0\mid w^{(t)}, w_k).$$
:eqlabel:`eq-negative-sample-conditional-prob`

设 $i_t$ 和 $h_k$ 分别为文本序列中时间步 $t$ 处的词 $w^{(t)}$ 和噪声词 $w_k$ 的索引。:eqref:`eq-negative-sample-conditional-prob` 中条件概率的对数损失为

$$
\begin{aligned}
-\log P(w^{(t+j)} \mid w^{(t)})
=& -\log P(D=1\mid w^{(t)}, w^{(t+j)}) - \sum_{k=1,\ w_k \sim P(w)}^K \log P(D=0\mid w^{(t)}, w_k)\\
=&-  \log\, \sigma\left(\mathbf{u}_{i_{t+j}}^\top \mathbf{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\left(1-\sigma\left(\mathbf{u}_{h_k}^\top \mathbf{v}_{i_t}\right)\right)\\
=&-  \log\, \sigma\left(\mathbf{u}_{i_{t+j}}^\top \mathbf{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\sigma\left(-\mathbf{u}_{h_k}^\top \mathbf{v}_{i_t}\right).
\end{aligned}
$$

我们可以看到，现在每次训练步骤的梯度计算成本与词典大小无关，而是线性依赖于 $K$。当设置超参数 $K$ 为较小值时，负采样每次训练步骤的梯度计算成本较小。

## 层次化 Softmax

作为一种替代的近似训练方法，*层次化 softmax* 使用二叉树数据结构，如 :numref:`fig_hi_softmax` 所示，其中树的每个叶节点代表词典 $\mathcal{V}$ 中的一个词。

![层次化 softmax 用于近似训练，其中树的每个叶节点代表词典中的一个词。](../img/hi-softmax.svg)
:label:`fig_hi_softmax`

设 $L(w)$ 为从根节点到表示词 $w$ 的叶节点路径上的节点数（包括两端）。设 $n(w,j)$ 为该路径上的第 $j$ 个节点，其上下文词向量为 $\mathbf{u}_{n(w, j)}$。例如，在 :numref:`fig_hi_softmax` 中 $L(w_3) = 4$。层次化 softmax 将 :eqref:`eq_skip-gram-softmax` 中的条件概率近似为

$$P(w_o \mid w_c) = \prod_{j=1}^{L(w_o)-1} \sigma\left( [\![  n(w_o, j+1) = \textrm{leftChild}(n(w_o, j)) ]\!] \cdot \mathbf{u}_{n(w_o, j)}^\top \mathbf{v}_c\right),$$

其中函数 $\sigma$ 定义见 :eqref:`eq_sigma-f`，$\textrm{leftChild}(n)$ 是节点 $n$ 的左子节点：如果 $x$ 为真，则 $[\![x]\!] = 1$；否则 $[\![x]\!] = -1$。

为了说明，我们计算在 :numref:`fig_hi_softmax` 中给定词 $w_c$ 生成词 $w_3$ 的条件概率。这需要词 $w_c$ 的词向量 $\mathbf{v}_c$ 与从根到 $w_3$ 的路径（:numref:`fig_hi_softmax` 中粗体路径）上的非叶节点向量之间的点积，该路径依次经过左、右、左：

$$P(w_3 \mid w_c) = \sigma(\mathbf{u}_{n(w_3, 1)}^\top \mathbf{v}_c) \cdot \sigma(-\mathbf{u}_{n(w_3, 2)}^\top \mathbf{v}_c) \cdot \sigma(\mathbf{u}_{n(w_3, 3)}^\top \mathbf{v}_c).$$

由于 $\sigma(x)+\sigma(-x) = 1$，基于任何词 $w_c$ 生成词典 $\mathcal{V}$ 中所有词的条件概率之和为 1：

$$\sum_{w \in \mathcal{V}} P(w \mid w_c) = 1.$$
:eqlabel:`eq_hi-softmax-sum-one`

幸运的是，由于二叉树结构的原因，$L(w_o)-1$ 的数量级为 $\mathcal{O}(\textrm{log}_2|\mathcal{V}|)$，当词典大小 $\mathcal{V}$ 非常大时，使用层次化 softmax 每次训练步骤的计算成本比没有近似训练的情况显著降低。

## 总结

* 负采样通过考虑既包含正例又包含负例的相互独立事件来构建损失函数。训练的计算成本线性依赖于每一步的噪声词数量。
* 层次化 softmax 使用从根节点到叶节点的路径来构建损失函数。训练的计算成本依赖于词典大小的对数。

## 练习

1. 在负采样中如何采样噪声词？
1. 验证 :eqref:`eq_hi-softmax-sum-one` 成立。
1. 如何分别使用负采样和层次化 softmax 来训练连续词袋模型？

[讨论](https://discuss.d2l.ai/t/382)